
 MSCS_634_Lab_4
 Roshan Gautam
 University of Cumberlands

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.pipeline import Pipeline
import warnings
warnings.filterwarnings('ignore')

# Set style for plots
plt.style.use('default')
sns.set_palette("husl")

# Step 1: Data Preparation
print("=== STEP 1: DATA PREPARATION ===")
diabetes = load_diabetes()
X, y = diabetes.data, diabetes.target

# Convert to DataFrame for better visualization
feature_names = diabetes.feature_names
X_df = pd.DataFrame(X, columns=feature_names)
y_df = pd.Series(y, name='target')

print(f"Dataset shape: {X_df.shape}")
print(f"Target shape: {y_df.shape}")
print(f"\nFeature names: {list(feature_names)}")
print(f"\nTarget statistics:")
print(y_df.describe())

# Check for missing values
print(f"\nMissing values in features: {X_df.isnull().sum().sum()}")
print(f"Missing values in target: {y_df.isnull().sum()}")

# Data distribution
fig, axes = plt.subplots(2, 5, figsize=(20, 8))
axes = axes.ravel()

for i, feature in enumerate(feature_names):
    axes[i].hist(X_df[feature], bins=20, alpha=0.7)
    axes[i].set_title(f'{feature}')
    axes[i].set_xlabel('Value')
    axes[i].set_ylabel('Frequency')

plt.tight_layout()
plt.show()

# Target distribution
plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.hist(y, bins=20, alpha=0.7)
plt.title('Target Distribution')
plt.xlabel('Target Value')
plt.ylabel('Frequency')

plt.subplot(1, 2, 2)
plt.boxplot(y)
plt.title('Target Boxplot')
plt.ylabel('Target Value')
plt.show()

# Correlation matrix
plt.figure(figsize=(12, 10))
correlation_matrix = X_df.corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0)
plt.title('Feature Correlation Matrix')
plt.show()

# Step 2: Linear Regression (Simple)
print("\n=== STEP 2: SIMPLE LINEAR REGRESSION ===")

# Use the feature with highest correlation to target
correlations = X_df.corrwith(y_df)
best_feature_idx = np.abs(correlations).idxmax()
print(f"Selected feature for simple regression: {best_feature_idx}")
print(f"Correlation with target: {correlations[best_feature_idx]:.3f}")

X_simple = X_df[[best_feature_idx]]
X_train_simple, X_test_simple, y_train, y_test = train_test_split(
    X_simple, y, test_size=0.2, random_state=42
)

# Train simple linear regression
lr_simple = LinearRegression()
lr_simple.fit(X_train_simple, y_train)
y_pred_simple = lr_simple.predict(X_test_simple)

# Calculate metrics
def calculate_metrics(y_true, y_pred, model_name):
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_true, y_pred)
    
    print(f"{model_name} Metrics:")
    print(f"  MAE: {mae:.3f}")
    print(f"  MSE: {mse:.3f}")
    print(f"  RMSE: {rmse:.3f}")
    print(f"  R²: {r2:.3f}")
    
    return {'MAE': mae, 'MSE': mse, 'RMSE': rmse, 'R2': r2}

simple_metrics = calculate_metrics(y_test, y_pred_simple, "Simple Linear Regression")

# Visualize simple regression
plt.figure(figsize=(10, 6))
plt.scatter(X_test_simple, y_test, alpha=0.6, label='Actual')
plt.scatter(X_test_simple, y_pred_simple, alpha=0.6, label='Predicted')
plt.xlabel(best_feature_idx)
plt.ylabel('Target')
plt.title('Simple Linear Regression: Actual vs Predicted')
plt.legend()
plt.show()

# Step 3: Multiple Regression
print("\n=== STEP 3: MULTIPLE REGRESSION ===")

X_train_multi, X_test_multi, y_train_multi, y_test_multi = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Train multiple regression
lr_multi = LinearRegression()
lr_multi.fit(X_train_multi, y_train_multi)
y_pred_multi = lr_multi.predict(X_test_multi)

multi_metrics = calculate_metrics(y_test_multi, y_pred_multi, "Multiple Linear Regression")

# Feature importance
feature_importance = pd.DataFrame({
    'feature': feature_names,
    'coefficient': lr_multi.coef_
}).sort_values('coefficient', key=abs, ascending=False)

print(f"\nFeature Importance (Coefficients):")
print(feature_importance)

# Visualize multiple regression
plt.figure(figsize=(10, 6))
plt.scatter(y_test_multi, y_pred_multi, alpha=0.6)
plt.plot([y_test_multi.min(), y_test_multi.max()], [y_test_multi.min(), y_test_multi.max()], 'r--', lw=2)
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.title('Multiple Linear Regression: Actual vs Predicted')
plt.show()

# Step 4: Polynomial Regression
print("\n=== STEP 4: POLYNOMIAL REGRESSION ===")

# Test different polynomial degrees
degrees = [2, 3, 4]
poly_results = {}

for degree in degrees:
    print(f"\nPolynomial Degree {degree}:")
    
    # Create polynomial features
    poly_features = PolynomialFeatures(degree=degree, include_bias=False)
    X_poly = poly_features.fit_transform(X)
    
    # Split data
    X_train_poly, X_test_poly, y_train_poly, y_test_poly = train_test_split(
        X_poly, y, test_size=0.2, random_state=42
    )
    
    # Train polynomial regression
    lr_poly = LinearRegression()
    lr_poly.fit(X_train_poly, y_train_poly)
    y_pred_poly = lr_poly.predict(X_test_poly)
    
    # Calculate metrics
    poly_metrics = calculate_metrics(y_test_poly, y_pred_poly, f"Polynomial (degree {degree})")
    poly_results[degree] = {
        'model': lr_poly,
        'metrics': poly_metrics,
        'y_pred': y_pred_poly,
        'y_test': y_test_poly
    }

# Visualize polynomial regression results
fig, axes = plt.subplots(1, len(degrees), figsize=(15, 5))
for i, degree in enumerate(degrees):
    result = poly_results[degree]
    axes[i].scatter(result['y_test'], result['y_pred'], alpha=0.6)
    axes[i].plot([result['y_test'].min(), result['y_test'].max()], 
                 [result['y_test'].min(), result['y_test'].max()], 'r--', lw=2)
    axes[i].set_xlabel('Actual Values')
    axes[i].set_ylabel('Predicted Values')
    axes[i].set_title(f'Polynomial Degree {degree}\nR² = {result["metrics"]["R2"]:.3f}')

plt.tight_layout()
plt.show()

# Step 5: Ridge and Lasso Regression
print("\n=== STEP 5: REGULARIZATION (RIDGE & LASSO) ===")

# Test different alpha values
alphas = [0.1, 1.0, 10.0, 100.0]
ridge_results = {}
lasso_results = {}

print("Ridge Regression:")
for alpha in alphas:
    ridge = Ridge(alpha=alpha, random_state=42)
    ridge.fit(X_train_multi, y_train_multi)
    y_pred_ridge = ridge.predict(X_test_multi)
    
    ridge_metrics = calculate_metrics(y_test_multi, y_pred_ridge, f"Ridge (α={alpha})")
    ridge_results[alpha] = {
        'model': ridge,
        'metrics': ridge_metrics,
        'y_pred': y_pred_ridge
    }
    print()

print("Lasso Regression:")
for alpha in alphas:
    lasso = Lasso(alpha=alpha, random_state=42, max_iter=2000)
    lasso.fit(X_train_multi, y_train_multi)
    y_pred_lasso = lasso.predict(X_test_multi)
    
    lasso_metrics = calculate_metrics(y_test_multi, y_pred_lasso, f"Lasso (α={alpha})")
    lasso_results[alpha] = {
        'model': lasso,
        'metrics': lasso_metrics,
        'y_pred': y_pred_lasso
    }
    print()

# Visualize regularization effects
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Ridge regression comparison
axes[0, 0].plot(alphas, [ridge_results[a]['metrics']['R2'] for a in alphas], 'o-', label='Ridge')
axes[0, 0].plot(alphas, [lasso_results[a]['metrics']['R2'] for a in alphas], 's-', label='Lasso')
axes[0, 0].set_xscale('log')
axes[0, 0].set_xlabel('Alpha')
axes[0, 0].set_ylabel('R² Score')
axes[0, 0].set_title('R² Score vs Alpha')
axes[0, 0].legend()
axes[0, 0].grid(True)

axes[0, 1].plot(alphas, [ridge_results[a]['metrics']['RMSE'] for a in alphas], 'o-', label='Ridge')
axes[0, 1].plot(alphas, [lasso_results[a]['metrics']['RMSE'] for a in alphas], 's-', label='Lasso')
axes[0, 1].set_xscale('log')
axes[0, 1].set_xlabel('Alpha')
axes[0, 1].set_ylabel('RMSE')
axes[0, 1].set_title('RMSE vs Alpha')
axes[0, 1].legend()
axes[0, 1].grid(True)

# Best Ridge and Lasso predictions
best_ridge_alpha = min(alphas, key=lambda a: ridge_results[a]['metrics']['RMSE'])
best_lasso_alpha = min(alphas, key=lambda a: lasso_results[a]['metrics']['RMSE'])

axes[1, 0].scatter(y_test_multi, ridge_results[best_ridge_alpha]['y_pred'], alpha=0.6)
axes[1, 0].plot([y_test_multi.min(), y_test_multi.max()], [y_test_multi.min(), y_test_multi.max()], 'r--', lw=2)
axes[1, 0].set_xlabel('Actual Values')
axes[1, 0].set_ylabel('Predicted Values')
axes[1, 0].set_title(f'Best Ridge (α={best_ridge_alpha})\nR² = {ridge_results[best_ridge_alpha]["metrics"]["R2"]:.3f}')

axes[1, 1].scatter(y_test_multi, lasso_results[best_lasso_alpha]['y_pred'], alpha=0.6)
axes[1, 1].plot([y_test_multi.min(), y_test_multi.max()], [y_test_multi.min(), y_test_multi.max()], 'r--', lw=2)
axes[1, 1].set_xlabel('Actual Values')
axes[1, 1].set_ylabel('Predicted Values')
axes[1, 1].set_title(f'Best Lasso (α={best_lasso_alpha})\nR² = {lasso_results[best_lasso_alpha]["metrics"]["R2"]:.3f}')

plt.tight_layout()
plt.show()

# Feature selection comparison
best_ridge_model = ridge_results[best_ridge_alpha]['model']
best_lasso_model = lasso_results[best_lasso_alpha]['model']

ridge_coefs = pd.DataFrame({
    'feature': feature_names,
    'ridge_coef': best_ridge_model.coef_,
    'lasso_coef': best_lasso_model.coef_
})

print(f"\nCoefficient Comparison (Ridge α={best_ridge_alpha}, Lasso α={best_lasso_alpha}):")
print(ridge_coefs.round(3))

# Step 6: Model Comparison and Analysis
print("\n=== STEP 6: MODEL COMPARISON ===")

# Compile all results
all_results = {
    'Simple Linear': simple_metrics,
    'Multiple Linear': multi_metrics,
    'Polynomial (deg=2)': poly_results[2]['metrics'],
    'Polynomial (deg=3)': poly_results[3]['metrics'],
    'Polynomial (deg=4)': poly_results[4]['metrics'],
    f'Ridge (α={best_ridge_alpha})': ridge_results[best_ridge_alpha]['metrics'],
    f'Lasso (α={best_lasso_alpha})': lasso_results[best_lasso_alpha]['metrics']
}

# Create comparison DataFrame
comparison_df = pd.DataFrame(all_results).T
print("Model Performance Comparison:")
print(comparison_df.round(3))

# Visualize comparison
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

metrics = ['MAE', 'MSE', 'RMSE', 'R2']
for i, metric in enumerate(metrics):
    ax = axes[i//2, i%2]
    values = comparison_df[metric]
    bars = ax.bar(range(len(values)), values)
    ax.set_xticks(range(len(values)))
    ax.set_xticklabels(values.index, rotation=45, ha='right')
    ax.set_ylabel(metric)
    ax.set_title(f'{metric} Comparison')
    
    # Highlight best performance
    if metric == 'R2':
        best_idx = values.idxmax()
        best_pos = list(values.index).index(best_idx)
        bars[best_pos].set_color('gold')
    else:
        best_idx = values.idxmin()
        best_pos = list(values.index).index(best_idx)
        bars[best_pos].set_color('gold')

plt.tight_layout()
plt.show()

# Summary insights
print("\n=== KEY INSIGHTS ===")
best_r2_model = comparison_df['R2'].idxmax()
best_rmse_model = comparison_df['RMSE'].idxmin()

print(f"Best R² Score: {best_r2_model} ({comparison_df.loc[best_r2_model, 'R2']:.3f})")
print(f"Best RMSE: {best_rmse_model} ({comparison_df.loc[best_rmse_model, 'RMSE']:.3f})")

print(f"\nPerformance Analysis:")
print(f"1. Multiple regression significantly outperformed simple regression")
print(f"2. Polynomial regression showed {'improvement' if comparison_df.loc['Polynomial (deg=2)', 'R2'] > comparison_df.loc['Multiple Linear', 'R2'] else 'mixed results'} over linear models")
print(f"3. Regularization helped {'control overfitting' if best_r2_model in ['Ridge', 'Lasso'] else 'but did not improve over polynomial models'}")
print(f"4. Lasso regression selected {np.sum(best_lasso_model.coef_ != 0)} out of {len(feature_names)} features")

print(f"\nDataset Insights:")
print(f"- Features show moderate correlations with target variable")
print(f"- Most important feature: {feature_importance.iloc[0]['feature']}")
print(f"- Regularization revealed feature importance patterns")
print(f"- Model complexity vs performance trade-offs observed")